<a href="https://colab.research.google.com/github/JJang-likit/GoingDeeper_Aiffel/blob/main/Going_Deeper1_%EB%B0%B1%EB%B3%B8_%EB%84%A4%ED%8A%B8%EC%9B%8C%ED%81%AC_%EA%B5%AC%EC%A1%B0_%EC%83%81%EC%84%B8%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Going Deeper 1. 백본 네트워크 구조 상세분석

### 1-1. 딥러닝 논문의 구조
- 초록(abstract) : 아이디어를 제안하는 방식과 학계에 이 논문이 기여하는 점을 요약한다.
- 서론(introduction), 관련 연구(related work) : 논문 내용과 제안하는 방법에 관한 이론 설명을 기술한다.
- 실험(experiments) : 소개한 방법을 확인하기 위한 실험 셋팅과 결과를 기술한다.
- 결론(conclusion) : 연구 내용 요약과 추가적인 연구방향을 제시한다.
- 참고문헌(reference), 부록(appendix) : 마지막에 참고한 문헌과 인용한 논문들의 리스트 그리고 본문에서 미처 설명하지 못한 추가적인 설명을 포함한다.

### 1-2. 논문의 형식 속에 담고자 하는 논리 구조
- 이전까지의 연구가 해결하지 못했던 문제의식
- 이 문제를 해결하기 위한 그동안의 다른 시도들
- 이 문제에 대한 이 논문만의 독창적인 시도
- 그러한 시도가 가져온 차별화된 성과

## 2. ResNet의 핵심 개념과 그 효과

### 2-1. ResNet 논문의 문제의식
- 서론을 통해서 ResNet 논문이 제기하고 있는 문제는 딥러닝 모델의 레이어를 깊이 쌓으면 항상 성능이 좋아지는가 하는 것이다.
- 이 문제에 대해서는 레이어를 깊이 쌓았을 때 생길 수 있는 Vanishing/Exploding Gradient 문제가 있지만 여기에 대해서는 이미 몇 가지 대응 방법이 있다.
- Vanishing/Exploding Gradient 문제를 해결하기 위한 방법으로는 **normalized initialization / intermediate normalization layers** 가 있다.
- 가장 눈에 띄는 키워드는 **Degradation Problem**이다.
- 이것은 모델의 수렴을 방해하는 Vanishing/Exploding Gradient 문제와 다르게 레이어를 깊에 쌓았을 때 모델이 수렴하고 있음에도 불구하고 발생하는 문제를 의미한다.
- Degradation Problem이 생기는 가장 큰 원인은 오버피팅이 아닌 네트워크 구조상 레이어를 깊이 쌓을수록 최적화가 잘 안되기 때문이다.

### 2-2. ResNet 논문이 제시한 솔루션 : Residual Block
- ResNet은 깊은 네트워크의 학습이 어려운 점을 해결하기 위해서 레이어의 입력값을 활용하여 레이어가 residual function(잔차 함수)를 학습하도록 한다.

## 3. DenseNet - Connection을 촘촘히
- DenseNet은 ResNet의 shortcut connection을 마치 fully connected layer처럼 촘촘히 가지도록 하면 더욱 성능 개선 효과가 클 것이라고 예상하고 이를 실험으로 입증하였다.

### 3-1. Dense Connectivity
- 일반적인 컨볼루션 네트워크가 L개의 레이어에 대해 각 레이어 간 하나씩의 연결, 즉 총 L개의 연결을 갖는 것과 달리 DenseNet의 기본 블록은 L개의 레이어가 있을 때 레이어 간 L(L + 1)/2 개의 직접적인 연결을 만든다.

### 3-2. Growth Rate
- 특성 맵을 더해주던 ResNet과 달리 DenseNet에서는 특성 맵을 채널 방향으로 쌓아서 사용한다.
- 입력값의 채널 수가 4인 경우 두번째 레이어는 첫번째 레이어의 출력값을 채널을 더해 8이 되고 세번째 레이어는 12개, 네번재 레이어는 16개의 특성 맵을 입력받는다.
- 입력값의 채널이 4로 시작했지만 진행할수록 특성 맵의 크기가 매우 커지는 것을 확인할 수 있다.

In [2]:
# Dense Block 내의 각 레이어 output의 channel을 계산하는 함수

def get_channel_list():
  channel_list = []
  input_channel = 32
  growth_rate = 32
  for i in range(12):
    channel_list.append(input_channel+growth_rate*i)

  return channel_list

get_channel_list()

[32, 64, 96, 128, 160, 192, 224, 256, 288, 320, 352, 384]

## 4. SENet - 어떤 특성이 중요할까?
- SENet은 Squeeze_and_Excitation Networks의 줄임말이다.
- 일반적인 CNN은 입력에 대해서 컨볼루션 필터를 필터 사이즈에 따라 적용한다.
- 하지만 SqueezeNet에서는 이 때 채널 방향으로 global average pooling을 적용, 압축된 정보를 활용하여 중요한 채널이 활성화되도록 한다.

### 4-1. Squeeze
- squeeze는 말 그대로 특성에서 중요한 정보를 짜내는 과정이다.
- 특정 맵의 채널에서 어느 채널이 중요한지 정보를 만들기 위해서는 우선 채널에 따른 정보를 압축해서 가져와야 한다.
- 채널별로 정보를 압축하기 위해서는 풀링(pooling)기법을 사용한다.
- 커널 영역에 대해 최댓값만 남기는 것이 Max Pooling, 평균값을 남기는 것이 Average Pooling이다.

### 4-2. Excitate
- 채널별 정보를 짜내고 나면 이제 채널별 정보에 따라 어떤 채널을 강조해야 할지 판단한다.
- 계산된 벡터를 기존의 특성 맵에 채널에 따라 곱해주어 중요한 채널이 활성화 되도록 만들어준다.

## 5. 모델 최적화하기 - Neural Architecture Search
- 우리가 딥러닝 모델의 파라미터를 최적화해 왔듯이 모델의 구조 자체도 최적화를 진행할 수 있다.
- 여러가지 네트워크 구조를 탐색하는 것을 아키텍쳐 탐색(architecture search)라고 하고 그 중 신경망을 사용해서 모델의 구조를 탐색하는 접근 방법을 NAS(neural architecture search)라고 한다.

### 5-1. NASNet
- NASNet은 머신 러닝에서 그리드 탐색(grid search) 등으로 실험과 모델 셋팅(config)를 비교하기 위한 자동화된 방법을 사용한다.
- 그리드 탐색은 모든 조합을 실험해보는 것을 의미한다.
- 하지만 그리드 탐색은 모델에서 바꿔볼 수 있는 구성의 종류가 매우 많아서 학습이 오래 걸리는 딥러닝에서는 적합하지 않다.
- NASNet은 하이퍼파라미터를 조정하면서 최적의 성능을 내도록 하는 방법으로 고안되었다.
- 아키텍쳐 탐색을 하는 동안 강화학습 모델은 신경망의 구성을 일종의 변수로 조정하면서 최적의 성능을 내도록 한다.
- 레이어의 세부 구성, CNN의 필터 크기, 채널의 개수, connection 등이 조정할 수 있는 변수가 된다.

### 5-2. Convolution cell
- 레이어 하나마다 하이퍼 파라미터를 조절한다면 탐색 공간이 무지막지 방대해진다.
- NASNet에서는 탐색 공간을 줄이기 위해 모듈(cell) 단위의 최적화를 하고 그 모듈을 조합하는 방식을 채택한다.
- convolution cell은 normal cell과 reduction cell로 구분된다.
- normal cell은 특성맵의 가로, 세로가 유지되도록 stride를 1로 고정한다.
- reduction cell은 stride를 1 또는 2로 가져가서 특성 맵의 크기가 줄어들 수 있도록 한다.
- 이 두가지 cell을 조합해서 최종 결과 네트워크인 NASNet을 만들어낸다.

## 6. 모델 최적화하기 - EfficientNet
- EfficientNet은 기존 모델들의 오류율을 뛰어넘고 모델의 크기인 Number of Parameters도 최적화 시켰다.
- EfficientNet는 우리가 이미지에 주로 사용하는 CNN을 효율적으로 사용할 수 있도록 네트워크의 형태를 조정할 수 있는 width, depth, resolution 세가지 요소에 집중한다.
- width는 CNN의 채널에 해당한다. 채널을 늘려줄수록 CNN의 파라미터와 특성을 표현하는 차원의 크기를 키울 수 있다.
- depth는 네트워크의 깊이이다.
- 마지막으로 resolution은 입력값을 너비(w)와 높이(h)의 크기를 의미한다. 입력이 클수록 정보가 많아져 성능이 올라갈 여지가 생기지만 레이어 사이의 특성 맵이 커지는 단점이 있다.